In [1]:
import pandas as pd
from pathlib import Path
import rdkit.Chem as Chem
from rdkit.Chem import Mol
from tqdm import tqdm
import requests

import torch
from torch_geometric.data import HeteroData
from torch.nn.functional import pairwise_distance

/Users/joschka/mambaforge/envs/kinodata/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from rdkit import RDLogger
RDLogger.DisableLog("rdApp.*")

In [3]:
RAW_DATA = Path() / "data" / "raw"

#### Read Kindodata data

In [4]:
df = pd.read_csv(RAW_DATA / "data_clean.csv", index_col="ident")
df.columns

Index(['activities.standard_type', 'activities.standard_units',
       'activities.standard_value', 'assays.chembl_id',
       'assays.confidence_score', 'component_sequences.sequence',
       'compound_structures.canonical_smiles',
       'compound_structures.standard_inchi', 'docking.posit_probability',
       'docking.runtime', 'docking.score', 'docking.similar_kinase_chain',
       'docking.similar_kinase_pdb', 'docking.similar_ligand_pdb',
       'docs.authors', 'docs.chembl_id', 'docs.klifs_kinase_id',
       'docs.klifs_structure_id', 'docs.year', 'molecule_dictionary.chembl_id',
       'molecule_dictionary.max_phase', 'structure_ID',
       'target_dictionary.chembl_id', 'target_dictionary.kinase',
       'target_dictionary.species', 'target_dictionary.uniprot_id'],
      dtype='object')

#### Assign ligand PDB filepaths

In [5]:
ligand_pdb_files = {
    int(fp.stem.split("_")[0]): fp for fp in (RAW_DATA / "pdbs" / "ligand").iterdir()
}
next(iter(ligand_pdb_files.items())), len(ligand_pdb_files)

((23120667, PosixPath('data/raw/pdbs/ligand/23120667_ligand.pdb')), 155718)

In [6]:
df["ligand_pdb_file"] = [ligand_pdb_files[ident] if ident in ligand_pdb_files else None for ident in df.index]

# sanity check
assert df["ligand_pdb_file"].notna().all()
df["ligand_pdb_file"].head()

ident
16291323    data/raw/pdbs/ligand/16291323_ligand.pdb
16306943    data/raw/pdbs/ligand/16306943_ligand.pdb
16264754    data/raw/pdbs/ligand/16264754_ligand.pdb
16340050    data/raw/pdbs/ligand/16340050_ligand.pdb
16340956    data/raw/pdbs/ligand/16340956_ligand.pdb
Name: ligand_pdb_file, dtype: object

#### Get pocket info from klifs

In [7]:
# NOTE one row is missing a structure ID
df["structure_ID"].isna().sum()

1

In [8]:
# drop those
df = df[df["structure_ID"].notna()]

In [9]:
# for some reason pandas reads structure ID as a float
df["structure_ID"] = df["structure_ID"].astype(int)

In [10]:
# get pocket mol2 files
pbar = tqdm(df.iterrows(), total=len(df))
for ident, row in pbar:
    structure_id = row["structure_ID"]
    fp = RAW_DATA / "mol2" / "pocket" / f"{structure_id}_pocket.mol2"
    if fp.exists():
        continue
    resp = requests.get(
        "https://klifs.net/api/structure_get_pocket", params={"structure_ID": structure_id}
    )
    if resp.ok:
        fp.write_bytes(resp.content)

100%|██████████| 110274/110274 [00:08<00:00, 13399.50it/s]


In [11]:
pocket_mol2_files = {
    int(fp.stem.split("_")[0]): fp for fp in (RAW_DATA / "mol2" / "pocket").iterdir()
}
df["pocket_mol2_file"] = [pocket_mol2_files[row["structure_ID"]] for _,row in df.iterrows()]

In [12]:
df[["pocket_mol2_file", "structure_ID"]].head()

,pocket_mol2_file,structure_ID
ident,,
16291323,data/raw/mol2/pocket/12757_pocket.mol2,12757
16306943,data/raw/mol2/pocket/14314_pocket.mol2,14314
16264754,data/raw/mol2/pocket/10689_pocket.mol2,10689
16340050,data/raw/mol2/pocket/10689_pocket.mol2,10689
16340956,data/raw/mol2/pocket/10689_pocket.mol2,10689


#### Create PyTorch Geometric data
this might take a while...

In [13]:
def atomic_numbers(mol) -> torch.LongTensor:
    z = torch.empty(mol.GetNumAtoms(), dtype=torch.long)
    for i, atom in enumerate(mol.GetAtoms()):
        z[i] = atom.GetAtomicNum()
    return z

def coordinates(mol) -> torch.FloatTensor:
    conf = mol.GetConformer()
    pos = conf.GetPositions()
    return torch.from_numpy(pos)

def add_atoms(mol, data, key):
    data[key].z = atomic_numbers(mol)
    data[key].pos = coordinates(mol)
    assert data[key].z.size(0) == data[key].pos.size(0)
    return data

In [14]:
data_list = []
n = 100 # debugging
removeHs = False
for ident, row in tqdm(df.sample(n).iterrows(), total=len(df)):
    data = HeteroData()
    
    ligand = Chem.MolFromPDBFile(str(row["ligand_pdb_file"]), removeHs=removeHs, sanitize=False)
    assert ligand is not None
    data = add_atoms(ligand, data, "ligand")
    
    pocket = Chem.rdmolfiles.MolFromMol2File(str(row["pocket_mol2_file"]), removeHs=removeHs, sanitize=False)
    assert pocket is not None
    data = add_atoms(pocket, data, "pocket")
   
    data.y = torch.tensor(row["activities.standard_value"]).view(1)
    
    data_list.append(data) 
    
    # TODO
    # edge information based on distance?
    ...

  0%|          | 100/110274 [00:02<44:50, 40.94it/s] 


In [15]:
data # structure and tensor shapes

HeteroData(
  y=[1],
  ligand={
    z=[57],
    pos=[57, 3]
  },
  pocket={
    z=[1397],
    pos=[1397, 3]
  }
)